<a href="https://colab.research.google.com/github/ModernOctave/Thermal-Image-Human-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Raw Dataset
Download and extract the raw dataset

In [ ]:
!wget -v https://adas-dataset-v2.flirconservator.com/dataset/full/FLIR_ADAS_v2.zip
!unzip -q FLIR_ADAS_v2.zip
!rm FLIR_ADAS_v2.zip

# Install Dependancies


In [ ]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 
import numpy as np
from PIL import Image
from pycocotools.coco import COCO
import random

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

# Create datasets
Create custom datasets required by the model

In [ ]:
def create_dataset(dataset_path):
  # Load the annotations
  coco_annotation_file_path = dataset_path+"coco.json"
  coco_annotation = COCO(annotation_file=coco_annotation_file_path)

  # Get all images
  img_ids = coco_annotation.getImgIds()[:3000]

  # Get category info
  cat_ids = coco_annotation.getCatIds()
  cat = dict(zip(cat_ids,[cat['name'] for cat in coco_annotation.loadCats(cat_ids)]))

  # Segregate images with and without humans
  human = []
  no_human = []

  for i, img_id in enumerate(img_ids):
    # Open img
    img_info = coco_annotation.loadImgs([img_id])[0]
    img_file_name = img_info["file_name"]
    im = np.array(Image.open(dataset_path+img_file_name))

    # Get all the annotations for the specified image.
    ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
    anns = coco_annotation.loadAnns(ann_ids)

    if 'person' in [cat[ann['category_id']] for ann in anns]:
      human.append(im)
    else:
      no_human.append(im)
    
    if i%(int(len(img_ids)/10)) == 0:
      print(str(int(i/len(img_ids)*100))+"%")

  half_size = min(len(human),len(no_human))
  images = human[:half_size]+no_human[:half_size]
  labels = list(np.ones(half_size))+list(np.zeros(half_size))
  dataset = list(zip(images, labels))
  random.shuffle(dataset)
  images, labels = zip(*dataset)
  images = (np.expand_dims(images, axis=-1)/255.).astype(np.float32)
  labels = np.array(labels).astype(np.int64)
  human = None
  no_human = None

  return images, labels

train_images, train_labels = create_dataset('images_thermal_train/')
print("Training dataset created!")
test_images, test_labels = create_dataset('images_thermal_val/')
print("Testing dataset created!")

# Model


## Create the model

In [ ]:
def build_cnn_model():
    cnn_model = tf.keras.Sequential([
                                     
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu, input_shape=(512, 640, 1)),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(64, activation=tf.nn.relu),

        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    
    return cnn_model
  
cnn_model = build_cnn_model()

# Initialize the model by passing some data through
cnn_model.predict(train_images[[0]])

# Compile the model
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the summary of the layers in the model.
print(cnn_model.summary())

## Train the model

In [ ]:
cnn_model = tf.keras.models.load_model("model.h5")

In [ ]:
cnn_model.fit(train_images, train_labels, epochs=5)

In [ ]:
cnn_model.save('model.h5')

## Test the model


In [ ]:
loss, accuracy = cnn_model.evaluate(test_images, test_labels, batch_size=1)